In [ ]:
# general imports
import matplotlib.pyplot as plt
import numpy as np
import numpy.typing as npt
import os
import pandas as pd

from functools import singledispatch

# dataset
from sklearn.datasets import fetch_kddcup99
from sklearn.preprocessing import StandardScaler

# pyspark module
from pyspark.rdd import RDD

# src module
from src.utils import kddSetup, sparkSetup
from src.kmeans import compute_centroidDistances, get_clusterId, get_minDistance, \
    kMeansParallel_init, kMeansPlusPlus_init, kMeansRandom_init, \
    miniBatchKMeans, naiveKMeans

In [ ]:
# creating the zipped environment if it doesn't already exist
! if [ ! -f "environment.tar.gz" ]; then venv-pack -o "environment.tar.gz" ; fi
# creating the zipped module src
! if [ -f "src.tar.gz" ]; then rm src.tar.gz ; fi
! tar -czf src.tar.gz src

In [ ]:
# starting the cluster
! $SPARK_HOME/sbin/start-all.sh

In [ ]:
# telling spark where to find the python binary
os.environ["PYSPARK_PYTHON"] = "./environment/bin/python"

In [ ]:
# creating a sparkSession
spark = sparkSetup("kMeans")
sc = spark.sparkContext
# exporting the src module
sc.addPyFile("src.tar.gz") 

### Naming conventions

The single datum is named as `datumName`, while the RDD that is a collection of one or more data is called `datumName_rdd`.

Example: `compute_clusterDistances` returns `clusterDistances` (i.e. a numpy array of distances between a point `x` and the `centroids`). 
The RDD that collects all the `clusterDistances` will be called `clusterDistances_rdd`, and here is a sample implementation of that:
```python
def compute_centroidDistances(x, centroids):
    return np.sum((centroids - x)**2, axis = 1)

# `data_rdd` is an RDD
# `centroids` is a numpy array
clusterDistances_rdd = data_rdd \
    .map(lambda x: compute_clusterDistances(x, centroids))
```

### Load and preprocess the dataset

In [ ]:
kdd_data, kdd_labels, entries_dict = kddSetup(standardize=True)

# get the number of clusters from kdd_labels
k = np.unique(kdd_labels).shape[0]

#parallelize
data_rdd = sc.parallelize([row for row in kdd_data])
data_rdd = data_rdd.persist()

In [ ]:
k = 15
l = k * 10
centroids = kMeansParallel_init(data_rdd, k, l)

In [ ]:
final_centroids = miniBatchKMeans(data_rdd, centroids, 10, 0.1)

In [ ]:
sc.stop()
spark.stop()

In [ ]:
# stopping the cluster
! $SPARK_HOME/sbin/stop-all.sh